<h2 style='color: orange'>Équipe: Louis & Emire</h2>

### 5. Comparaison des performances entre les méthodes linéaires vs méthodes non-linéaires pour la prédiction sur de vraies données transcriptomiques des cancers de TCGA.

# Workplan

1. Review litterature about the type of ML algorithms and their models
2. Data importation: `TCGA dataset`
3. Data preprocessing -> experience data
4. Algorithms/models configuration
5. Models training
6. Comparison with figures
   - 2 clusters containing maximum 4 figures each.
   - 1 classification plot for each algorithm (2 in total for each cluster)

In [ ]:
# Librairies
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import itertools
from collections import Counter
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from utils.random_color_generator import random_color_generator
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
from utils.DataSet import DataSet

# Data importation and pre-processing

data_set = DataSet('./data/TCGA_TPM_hv_subset.h5')

figures_directory = "./figures/project"

data = data_set.get_data('data', float)

genes = data_set.get_data('cols', str)
labels = data_set.get_data('labels', str)
rows = data_set.get_data('rows', str)
fig, axs = plt.subplots(2, 2, figsize = (20, 20))

### A. Pour la classification moléculaires entre la régression linéaire et un DNN, quelle méthode fonctionne le mieux et pourquoi?


#### Régression Logistique

In [ ]:
max_iter = 100
X_train, X_test, Y_train, Y_test = train_test_split(data.T, labels, test_size= 0.2)
log_clf = RidgeClassifier(alpha=1e-4, solver ="lsqr", max_iter = max_iter)
log_clf.fit(X_train, Y_train)

train_set_accuracy = round((np.mean(log_clf.predict(X_train) == Y_train)), 2) * 100
test_set_accuracy = round((np.mean(log_clf.predict(X_test) == Y_test)), 2) * 100

print(f"Accuracy on train set (n = {X_train.shape[0]}): {train_set_accuracy}%")
print(f"Accuracy on test set (n = {X_test.shape[0]}): {test_set_accuracy}%")
print(f"Number of errors : {(np.sum(log_clf.predict(X_test) != Y_test)) }")

Analysis

In [329]:
cm = confusion_matrix(log_clf.predict(X_test),  Y_test)

# fig = plt.figure(figsize = (10, 10))
axs[0,0].imshow(cm, cmap='Oranges', vmin=cm.min(), vmax=cm.max())
axs[0,0].set_xticks(np.arange(len(np.unique(labels))), labels = np.unique(labels), rotation = 45, ha='right')
axs[0,0].set_yticks(np.arange(len(np.unique(labels))), labels = np.unique(labels))
axs[0,0].set_xlabel("Predicted Labels")
axs[0,0].set_ylabel("True Labels")
axs[0,0].set_title("Ridge classifier confusion matrix on test set")

thresh = cm.max() / 1.5

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if cm[i, j] != 0:
        axs[0,0].text(j, i, "{:,}".format(cm[i, j]),
        horizontalalignment="center",
    color="white" if cm[i, j] > thresh else "black")

In [330]:
# Get predictions
predictions = log_clf.predict(X_test)

# Find indices where predictions and actual values differ
error_indices = np.where(predictions != Y_test)[0]

# plt.figure(figsize = (8, 8))
# Assuming your data is 2D, plot the first two features for simplicity
# Adjust this based on your actual data
axs[1,0].scatter(X_test[error_indices, 0], X_test[error_indices, 1], color='red', label='Errors')

# Optionally, plot correct predictions for comparison
correct_indices = np.where(predictions == Y_test)[0]
axs[1,0].scatter(X_test[correct_indices, 0], X_test[correct_indices, 1], color='green', alpha=0.5, label='Correct')

axs[1,0].set_xlabel('RC1')
axs[1,0].set_ylabel('RC2')
axs[1,0].set_title(f'Ridge classifier errors plot on the test set\nAccuracy : {test_set_accuracy}%, Iters = {max_iter}')
axs[1,0].legend()

#### DNN

Data

In [ ]:
lblbin = LabelBinarizer()

targets = lblbin.fit_transform(labels)

X, Y = torch.Tensor(data.T), torch.Tensor(targets)

X_train, X_test, Y_train, Y_test = [torch.Tensor(split) for split in train_test_split(data.T, targets, test_size = 0.2)]

Model

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, insize, outsize):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(insize, 100), # couche entrée
            nn.Linear(100, 100), # couche cahée
            nn.Linear(100, outsize)  # couche sortie
        )

    def train(self, nepochs = 1000, printstep = 100):
        mm = self.linear_relu_stack        
        optimizer = torch.optim.Adam(mm.parameters(), lr = 1e-4)
        tr_losses, tst_losses, tr_accs, tst_accs = [], [], [], [] 
        # boucle de training 
        for i in range(nepochs):
            #####
            optimizer.zero_grad() # required*
            out_tr = mm(X_train) # calcul 'avant'
            tr_error = nn.functional.cross_entropy(out_tr, Y_train)
            out_test = mm(X_test)
            tst_error = nn.functional.cross_entropy(out_test, Y_test)
            tr_acc = np.mean(np.array( out_tr.max(1).indices == Y_train.max(1).indices))
            tst_acc = np.mean(np.array( out_test.max(1).indices == Y_test.max(1).indices))
            [tr_losses.append(float(tr_error)),tst_losses.append(float(tst_error)),
            tr_accs.append(tr_acc), tst_accs.append(tst_acc) ]
            tr_error.backward() # calcul 'arriere' et mise a jour
            optimizer.step() # required*
            if i % printstep == 0 or i == nepochs - 1:
                print(f"{i} Loss Train : {round(float(tr_error) * 100,2)} - Acc: {round(tr_acc * 100,2)} \
                    - Test : {round(float(tst_error) * 100,2)} - Acc: {round(tst_acc * 100,2)} \
                        - errors: {np.sum(np.array( out_test.max(1).indices != Y_test.max(1).indices))}")
        return mm, tr_losses, tst_losses, np.array(tr_accs), np.array(tst_accs)

    

Training

In [ ]:
nni = NeuralNetwork(X_train.shape[1], Y_train.shape[1])
mm, tr_losses, tst_losses, tr_accs, tst_accs = nni.train(nepochs = 100, printstep = 100)

Analysis

Learning curves

In [331]:
axs[1, 1].cla()

markers_ = np.concatenate([['o',"v","^","<",">","8","p","s","h","D","P","X"] for i in range(10)])

steps = np.arange(len(tr_losses))

from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='green', lw=2),
                Line2D([0], [0], color='blue', lw=2),
                Line2D([0], [0], color='orange', lw=2, linestyle='--'),
                Line2D([0], [0], color='purple', lw=2, linestyle='--')]

# First plot (Losses) on axs[0, 1]
axs[1, 1].plot(steps, tr_losses, label="train (loss)", color='green')
axs[1, 1].plot(steps, tst_losses, label="test (loss)", color='blue')
axs[1, 1].plot(steps[-1], tr_losses[-1], 'go', markersize=5, label=f'({steps[-1]}, {tr_losses[-1]:.2f})')
axs[1, 1].plot(steps[-1], tst_losses[-1], 'bv', markersize=5, label=f'({steps[-1]}, {tst_losses[-1]:.2f})')
axs[1, 1].set_ylabel("Cross entropy loss")
axs[1, 1].set_xlabel("Epochs number")
axs[1, 1].legend(loc='center right', bbox_to_anchor=(1, 0.575))

# Second plot (Accuracies) on the same subplot, but using a secondary y-axis
sec_ax = axs[1, 1].twinx()
sec_ax.plot(steps, tr_accs * 100, label="train (accuracy)", color='orange')
sec_ax.plot(steps, tst_accs * 100, label="test (accuracy)", color='purple')
sec_ax.plot(steps[-1], tr_accs[-1] * 100, 'o', color='orange', markersize=5, label=f'({steps[-1]}, {tr_accs[-1]*100:.2f}%)')
sec_ax.plot(steps[-1], tst_accs[-1] * 100, 'v', color='purple', markersize=5, label=f'({steps[-1]}, {tst_accs[-1]*100:.2f}%)')
sec_ax.set_ylabel("Accuracy (%)")
sec_ax.set_ylim((0, 100))
sec_ax.legend(loc='center right', bbox_to_anchor=(1, 0.425))

# Set the title for the combined plot
axs[1, 1].set_title(f"Learning curves of DNN on classification of cancer type in TCGA data\nN={data.T.shape[1]}, N(train)={X_train.shape[0]}, N(test)={X_test.shape[0]}")

Text(0.5, 1.0, 'Learning curves of DNN on classification of cancer type in TCGA data\nN=15165, N(train)=8276, N(test)=2070')

Confusion matrix

In [335]:
cm = confusion_matrix(log_clf.predict(X_test),  Y_test)

axs[0,1].imshow(cm, cmap='Blues', vmin=cm.min(), vmax=cm.max())
axs[0,1].set_xticks(np.arange(len(np.unique(labels))), labels = np.unique(labels), rotation = 45, ha='right')
axs[0,1].set_yticks(np.arange(len(np.unique(labels))), labels = np.unique(labels))
axs[0,1].set_xlabel("Predicted Labels")
axs[0,1].set_ylabel("True Labels")
axs[0,1].set_title("DNN confusion matrix on test set")

thresh = cm.max() / 1.5

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if cm[i, j] != 0:
        axs[0,1].text(j, i, "{:,}".format(cm[i, j]),
        horizontalalignment="center",
    color="white" if cm[i, j] > thresh else "black")

Grouped

In [336]:
fig.savefig(f'{figures_directory}/project_question_a_grouped_figures.svg', format='svg')

#### B. Pour la régression entre la régression linéaire et un Auto-Encodeur, quelle méthode fonctionne le mieux et pourquoi?


#### C. Démontrez qu'une PCA 2D est équivalent à la couche interne bottleneck d'Auto-Encodeur linéaire.